In [2]:
import pandas as pd
import sys
import warnings
warnings.filterwarnings('ignore')


sys.path.append('../')
from src.cleaning import remove_nan
from src.cleaning import THRESHOLD
from src.cleaning import remove_all_outliers

data = pd.read_csv('../data/assignment_rev2.csv')
# we remove all columns with more than 80% (threhold) of NA values due they corrupt our results
data = remove_nan(data,threshold=THRESHOLD)


In [4]:
data.columns

Index(['id', 'ranking_score', 'agent_id', 'geography_name', 'sq_meters',
       'price', 'year_of_construction', 'floor', 'subtype', 'rooms',
       'no_of_bathrooms', 'no_of_wc', 'ad_type', 'living_rooms', 'kitchens',
       'balcony_area', 'lux_home', 'new_development', 'garage', 'furnished',
       'preserved', 'investment', 'unfinished', 'heating_under_floor',
       'renovated', 'requires_renovation', 'airy', 'bright', 'painted',
       'garden', 'ac', 'storage', 'balcony', 'internal_stairs', 'view',
       'secure_door', 'penthouse', 'alarm', 'corner', 'holiday_home',
       'student_home', 'night_power', 'neoclassic', 'double_glass',
       'for_commercial_use', 'fireplace', 'elevator', 'satellite',
       'solar_heater', 'attic', 'pool', 'playroom'],
      dtype='object')

In [3]:
#create a subset based on the interest we have
set_1 =  ['id','ranking_score','geography_name','sq_meters','price','year_of_construction','ad_type','floor','rooms','subtype']
subset = data[set_1]
subset = subset.dropna(axis=0,how='any')

# We create the price per square meter column
subset['price_per_sqrm'] = round(subset['price'] / subset['sq_meters'],0)

print("Before the outlier handling {:.2f} % of the original dataset is preserved".format(len(subset)/len(data)*100))
subset = remove_all_outliers(subset, not_count = ['year_of_contruction']).reset_index(drop=True)
print("\nAfter the removal of outliers {:.2f} % of the original dataset is preserved".format(len(subset)/len(data)*100))

Before the outlier handling 95.26 % of the original dataset is preserved

After the removal of outliers 91.10 % of the original dataset is preserved


### Anti-Corruption of dataset 

To avoid further corruption of our dataset, we will remove:
- everything that is under construction (as the data refers to this month)
- Any entry that has significanlty low price that cannot be realistic (<10)

<i>We pressume that those values indicate a negotiable price for the condo without defining the starting point of price</i>


In [4]:
subset = subset[subset['year_of_construction'] != 2155]
subset = subset[subset['price'] > 10]
subset['price'] = subset['price'].apply(int)

In [5]:
print("\nAfter the the final steps {:.2f} % of the original dataset is preserved".format(len(subset)/len(data)*100))


After the the final steps 87.69 % of the original dataset is preserved


In [6]:
display(subset.describe().round(0))

,id,ranking_score,sq_meters,price,year_of_construction,rooms,price_per_sqrm
count,13545.0,13545.0,13545.0,13545.0,13545.0,13545.0,13545.0
mean,41330985.0,119.0,180.0,513516.0,1992.0,3.0,2852.0
std,2084003.0,31.0,140.0,442933.0,20.0,1.0,1419.0
min,35000486.0,24.0,12.0,300.0,1901.0,0.0,4.0
25%,40463714.0,98.0,87.0,220000.0,1977.0,2.0,1905.0
50%,42131341.0,123.0,134.0,380000.0,1995.0,3.0,2632.0
75%,42848201.0,144.0,231.0,650000.0,2009.0,3.0,3488.0
max,43464103.0,183.0,3000.0,2600000.0,2020.0,10.0,16667.0


In [7]:
subset.head(3)

,id,ranking_score,geography_name,sq_meters,price,year_of_construction,ad_type,floor,rooms,subtype,price_per_sqrm
0,42911697,47.0,northern sub,567,200000,2005,simple,1,4,apartment,353.0
1,43018583,134.7,northern sub,642,550000,1992,simple,ground-floor,5,detached,857.0
2,39670968,71.0,northern sub,320,500000,2009,simple,ground-floor,2,detached,1562.0


~88% of our original dataset is a decent representation of the market at this point. 

We will save this dataset and move to the 1st part of the assignment to Notebook Part1.ipynb

In [8]:
subset.to_csv('../data/subset.csv', encoding='utf-8-sig',index=False)